# Converting Handwritten text into pdf and doc file
# Analyts the text to produce the 10 most repeated words
# use NLP to extract the summer of the text

In [1]:
# use google-vision to read the scan image
# save the file to pdf and doc file
# save the same file at that time to a string
# run the NLP on the text and find the summary
# produce the 10 most repeating words
# made a data fram contain the file number, the summary, and the 10 most repeated world

The purpose of the research is to made hand written text easy read; to make handwritten to easy to convert to pdf or doc file for easy submission for students.
Also, the NLP done on the handwritten text is to prodive a quick summary on what ever they user is about to read. 
Instead of using a human reviewer to go read over the text, computers can do that in miliseconds and provide a summary

The aim of this research is to produce and easy and quick way for people who love writting this down to convert their data into 
into required dociments types for their work.
Also, the project aims at producing an easy way for student to get the context and summary of what ever handwritten note they want to read.

In [86]:
# Importations of the need librarys
import cv2
import os, io
from google.cloud import vision
from google import type
import pandas as pd

from fpdf import FPDF
from docx import Document
import nltk


In [155]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
!pip install docx2pdf
from docx2pdf import convert

Error processing line 1 of C:\Users\Lucky Abolorunke\Anaconda3\lib\site-packages\vision-1.0.0-py3.8-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\Lucky Abolorunke\Anaconda3\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 553, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [88]:
# google cloud vision
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\Lucky Abolorunke\Downloads\ServiceAccountToken.json"

In [89]:
# to extract the images from the pictures 
# this function takes in a picture and returns the image, bu this time the color has be removed, 
# and everything would be in black and white

def ReadImages(file_name):
    
    with io.open(file_name, 'rb') as imgx:
        content = imgx.read()
    image = vision.Image(content=content)

    return image 

In [96]:
# the function ImageText takes in the directory of a picture and returns a text in the image in form a string

def ImageText(file_name):

    client = vision.ImageAnnotatorClient()

    img = ReadImages(file_name)
    response = client.document_text_detection(image=img)
    doc = response.full_text_annotation.text
    doc = doc.replace('\n', ' ')
    return doc     

In [99]:
# test the picture reader
f_n =r"C:\Users\Lucky Abolorunke\Desktop\2.JPG"
f = ImageText(f_n)
print(f)

Ad Creating De U commandment now comfossim a new Commission new consecration new contribution a new compensation w commitment without courage you can't do what God want you 38 SUBTOPIS: FOMPLING COMPREHENSIONE OF OUR TASK OUTSIDED HOMPLETE CONSECRATION OF OUR TEMPLE contourageous COMMITMENT 2 fils 2 summary Onez, Sono Compling eumprehension of our task-outside the temples Cimonclés aqus_9 luke lovs 2.7, 30,331 Kings 4u Luce aodem? nos gust prowbs aqui,t2 Buonany tour guest + Complete consecration of our temple us 29034 Actio + proverbs US 20 2 lguses nots 1905.25 10 Chronides 2ows 20,21 James 54519 EHT chronido 299 Matthew Busig mark Il con- 5 proverbs 34 Sconosis agus 20 'malachi gusi i corinthians Lost15 I Corronícies isvs29 nitilans puss -12 MS41 Lulce gus 38 agus 9. 


The following lines of code is use to summarize the tex and fine the 10 most repeated words

In [33]:
def wordCount(text):
    from string import punctuation
    word_count = {}
    stopwords = list(STOP_WORDS)
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
#     tokens = [token.text for token in doc]
    
    #puntuations
    punctuation = punctuation + "\n"
    for word in doc:
        if word.text.lower() not in stopwords:
            if word.text.lower() not in punctuation:
                if word.text not in word_count:
                    word_count[word.text] = 1
                else:
                    word_count[word.text] += 1
    return word_count        
    
    

In [56]:
def tenMostCommon(text):
    word_count = wordCount(text)
    # create a sorted list of the items in the dicitionary
    numbers = sorted(word_count.values())
    result = []
    
    count = 0
    index = len(numbers)-1
    while count < 10:
        for key in word_count.keys():
            if word_count[key] == numbers[index] and count < 10:
                result.append(key + ": "+str(numbers[index]))
                index -= 1
                count += 1
        
    return result   


In [109]:
# summarize
def Summarize(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    word_count = wordCount(text)
    max_frequency = max(word_count.values())    
    for word in word_count.keys():
        
        word_count[word] = word_count[word]/max_frequency
    sen_tokens = [sen for sen in doc.sents]
    
    # map the frequency of every sentence to the sentence
    sentence_scores = {}
    for sen in sen_tokens:
        for word in sen:
            if word.text.lower() in word_count.keys():
                if word.text.lower() not in sentence_scores.keys():
                    sentence_scores[sen] = word_count[word.text.lower()]
                else:
                    sentence_scores[sen] += word_count[word.text.lower()]
    select_length = int(len(sentence_scores)*0.3)

    summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    
    sumx = [word.text for word in summary]
    final_summary = " ".join(sumx)                
    #return the summary
    return final_summary

 

In [121]:
def nameFile(string):
    name = ""
    word = ""
    count = 0
    
    for char in string:
        if char == " " and count < 4:
            count += 1
            name += word
            word = ""
            if count <= 3:
                name += " "
        else:
            word += char
    return name        

In [165]:
# write to pdf
# the function take in two string type(text, summary), a list of strings, and then save a doc file to the PC
def writePDF(text, most_words, summary):
#     use the covert class from docxpdf module to convert a created doc file to pdf
    name = nameFile(summary)
    output = r"C:\Users\Lucky Abolorunke\Desktop\handwriting\{}.pdf".format(name)

    convert(writeDocument(text, most_words, summary), output)


In [177]:
# to write the document to file
# the function take in two string type(text, summary), a list of strings, and then save a doc file to the PC

def writeDocument(text, most_words, summary):
    document = Document()
    # this sets the converted handwritten note to the doc
    document.add_paragraph(text)
    
    # set a header that indicates the time most repeated words
    # level two size of header
    document.add_heading("Ten most repeated words", 2)
    document.add_paragraph(", ".join(most_words))
    
    # add another hearder for the summary
    document.add_heading("Summary of the note", 2)
    document.add_paragraph(summary)
    
    # use the first 4 words to save he file
    name = nameFile(summary)
    #save file to directory
    directory = r"C:\Users\Lucky Abolorunke\Desktop\handwriting\{}.docx".format(name)
    document.save(directory)
    return directory


In [178]:
def typeDoc():
    num = int(input("1.docx\n2.pdf\nEnter the number: "))
    return num
    

In [179]:

 def convertFile(file_name):
    text = ImageText(file_name)
    common_words = tenMostCommon(text)
    summary = Summarize(text)
    
    num = typeDoc()
    if num == 1:
        writeDocument(text, common_words, summary)
    else:
        writePDF(text, common_words, summary)
    print("Done!!!")
    

In [181]:
for index in range(1,26):
    directory = r"C:\Users\Lucky Abolorunke\Desktop\handwriting\photos\{}.jpg".format(index)
    convertFile(directory)


1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1
Done!!!
1.docx
2.pdf
Enter the number: 1


OSError: [Errno 22] Invalid argument: 'C:\\Users\\Lucky Abolorunke\\Desktop\\handwriting\\* Virtues of the.docx'

# 